In [1]:
import pandas as pd 
import numpy as np
!pip install folium

     |████████████████████████████████| 92kB 7.5MB/s eta 0:00:011


### IMPORTING THE TABLE FROM WIKIPEDIA AFTER GETTING IT AS A CSV FILE 

In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_abd1149ae2664d17ad4727bc555ff7f1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='WxERfwO9ihKUrCIRI5WNtueXe6_8t6Zk1JKUcfI51aVE',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_abd1149ae2664d17ad4727bc555ff7f1.get_object(Bucket='thebestclassifierbyahmedelrefaey-donotdelete-pr-sbsdlu3g56zwht',Key='table-1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# DATA CLEANING [deleting rows with Borough=Not Assigned]

In [3]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

In [4]:
df[df['Neighborhood']=='Not assigned']

,Postal code,Borough,Neighborhood


In [5]:

df.reset_index(inplace=True,drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### replacing "/" with "," 

In [6]:
df["Neighborhood"].replace(" / "," , ",regex=True,inplace =True)  
                          

In [7]:
df.shape

(103, 3)

### AFTER MULTIBLE TRIES TO GET THE CORDINATES using the Geocoder package I WILL USE THE TABLE SENT BY THE INSTRUCTOR

In [8]:

body = client_abd1149ae2664d17ad4727bc555ff7f1.get_object(Bucket='thebestclassifierbyahmedelrefaey-donotdelete-pr-sbsdlu3g56zwht',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dfcordinates = pd.read_csv(body)
dfcordinates.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### MERGING THE TWO TABLES

In [18]:

dfcordinates.rename(columns={"Postal Code":"Postal code"},inplace=True)
dfmerged=df.merge(dfcordinates, on="Postal code" )
dfmerged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [19]:
condition=dfmerged['Borough'].str.contains(pat = 'Toronto') 
dfmerged=dfmerged[condition]
dfmerged.reset_index(inplace=True,drop=True)


dfmerged = dfmerged.groupby('Neighborhood').mean().reset_index()
dfmerged.shape

(39, 3)

### getting the cordinates for toronto city

In [21]:

from geopy.geocoders import Nominatim 
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


## crating a toronto map with its neighborhoods

In [22]:
# create map of toronto using latitude and longitude values

import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfmerged['Latitude'], dfmerged['Longitude'], dfmerged['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

# foursquare credentials

In [23]:
CLIENT_ID = 'MBT4LMRUADMEP1UAE3H53UWI0AIOBTA0UHTHR0RRSFR5BQE0' # your Foursquare ID
CLIENT_SECRET = 'SLTAEWBCL5JZ50L2SNMQX3AAP5F5MMPNQFL4U5GYNJAG03BS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MBT4LMRUADMEP1UAE3H53UWI0AIOBTA0UHTHR0RRSFR5BQE0
CLIENT_SECRET:SLTAEWBCL5JZ50L2SNMQX3AAP5F5MMPNQFL4U5GYNJAG03BS


## getting the info of  each Neighborhood from foursquare 

In [24]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names=dfmerged['Neighborhood'],
                                   latitudes=dfmerged['Latitude'],
                                   longitudes=dfmerged['Longitude']
                                  )

Berczy Park
Brockton , Parkdale Village , Exhibition Place
Business reply mail Processing CentrE
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Central Bay Street
Christie
Church and Wellesley
Commerce Court , Victoria Hotel
Davisville
Davisville North
Dufferin , Dovercourt Village
First Canadian Place , Underground city
Forest Hill North & West
Garden District, Ryerson
Harbourfront East , Union Station , Toronto Islands
High Park , The Junction South
India Bazaar , The Beaches West
Kensington Market , Chinatown , Grange Park
Lawrence Park
Little Portugal , Trinity
Moore Park , Summerhill East
North Toronto West
Parkdale , Roncesvalles
Queen's Park , Ontario Provincial Government
Regent Park , Harbourfront
Richmond , Adelaide , King
Rosedale
Roselawn
Runnymede , Swansea
St. James Town
St. James Town , Cabbagetown
Stn A PO Boxes
Studio District
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
The Anne

In [27]:
torontodata=toronto_venues.groupby('Neighborhood').count()

### converting the categorical data to one coded

In [28]:
#toronto_venues# one hot encoding
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


In [30]:
cols = list(onehot.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('Neighborhood')) #Remove b from list

onehot = onehot[['Neighborhood']+cols] #Create new dataframe with columns in the order you want
onehot.head()


,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Berczy Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.017241,...,0.000000,0.0,0.0,0.017241,0.0,0.0,0.00,0.0,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.058824
3,"CN Tower , King and Spadina , Railway Lands , ...",0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000
4,Central Bay Street,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.014925,0.0,0.0,0.00,0.0,0.0,0.014925
5,Christie,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000
6,Church and Wellesley,0.0000,0.0000,0.000,0.0000,0.000,0.014085,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.028169
7,"Commerce Court , Victoria Hotel",0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.0,0.0,0.010000,...,0.000000,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.0,0.000000
8,Davisville,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.027778,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000
9,Davisville North,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000


## returnin the neighbourhood with the first 10 venues

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Italian Restaurant,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Nightclub,Pet Store,Climbing Gym,Restaurant,Burrito Place
2,Business reply mail Processing CentrE,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Light Rail Station
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Ice Cream Shop


### startin the clustering 

In [35]:
# set number of clusters
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

# add clustering labels

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merg = dfmerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merg.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Italian Restaurant,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bakery,Nightclub,Pet Store,Climbing Gym,Restaurant,Burrito Place
2,Business reply mail Processing CentrE,43.662744,-79.321558,0,Yoga Studio,Fast Food Restaurant,Comic Shop,Park,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Light Rail Station
3,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420,0,Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Plane,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique
4,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Ice Cream Shop


# creating the map after clustering 

In [37]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merg['Latitude'], toronto_merg['Longitude'], toronto_merg['Neighborhood'], toronto_merg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# thank you , good luck with your assignment